In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/competition/2022-AI-competition-Round1")
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['2022-08-06 Optimize ET.ipynb',
 '2022-08-17 Ensemble Module.ipynb',
 'competition_data',
 'html',
 'submission',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'model compare.ipynb',
 'Optuna Optimization.ipynb',
 '2022-08-06 Regressor model compare.ipynb',
 'catboost_info',
 'evaluator',
 '.git',
 '.gitignore',
 'README.md',
 'Updated CLF.ipynb',
 '2022-08-11 Models(XGB_ET) Optimization.ipynb',
 'GitHub Connection.ipynb']

In [2]:
!pip install -r evaluator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from evaluator.evaluator import Evaluator, Model

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")

### baseline

In [6]:
Evaluator(
    **Model(train_df, "et", "clf").get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.780244,0.788391,0.831849,0.809537,0.872887,0.302898,NaN
2,0.784308,0.775799,0.849684,0.811062,0.877114,0.305249,NaN
3,0.764927,0.766200,0.830804,0.797195,0.861757,0.314742,NaN
4,0.784308,0.802514,0.830352,0.816196,0.867259,0.305771,NaN
mean,0.778446,0.783226,0.835672,0.808498,0.869754,0.307165,1.0


### optimize

In [4]:
model = Model(train_df, "et", "clf")

In [7]:
initial_params = (
    ("n_estimators", "int", (200, 500)),
    ("max_depth", "int", (10, 70)),
    ("min_samples_split", "int", (2, 5)),
    ("min_samples_leaf", "static", 1),
    ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
    ("max_features", "float", (0.6, 0.8)),
    ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
    ("max_leaf_nodes", "int", (30, 200)),
    ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
)
model.optimize(initial_params, n_runs=2)

TypeError: ignored

In [ ]:
model = Model(train_df, "lgbm", "rgr")

In [ ]:
initial_params = (
    ("n_estimators", "static", 100),
    ("objective", "static", "binary"),
    ("metric", "static", "auc"),
    ("learning_rate", "log", (1e-5, 1.0)),
    ("num_leaves", "int", (300, 2000)),
)
model.optimize(initial_params, n_trials=20)

[I 2022-08-06 07:59:08,190] A new study created in memory with name: no-name-8bb7839b-bc15-4186-92b9-42c719035901
[I 2022-08-06 07:59:20,174] Trial 0 finished with value: 0.8508964743241424 and parameters: {'learning_rate': 0.0375382101697422, 'num_leaves': 1744}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:31,942] Trial 1 finished with value: 0.8251172008255213 and parameters: {'learning_rate': 0.012936354125886644, 'num_leaves': 1404}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:44,056] Trial 2 finished with value: 0.7989137223114446 and parameters: {'learning_rate': 0.004247809355124376, 'num_leaves': 1996}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:55,721] Trial 3 finished with value: 0.8059669513353841 and parameters: {'learning_rate': 0.005842855674677552, 'num_leaves': 1246}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 08:00:06,136] Trial 4 finished with value: 0.748021939779588 and param

0.8630226961242209
{'learning_rate': 0.3225475195598088, 'num_leaves': 729}


In [ ]:
optuna.visualization.plot_slice(model.optimizer.opt)

In [ ]:
Evaluator(
    **model.get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.763676,0.781081,0.804566,0.792649,0.839295,0.291786,NaN
2,0.764927,0.765399,0.819851,0.791690,0.840958,0.295799,NaN
3,0.761175,0.774473,0.804947,0.789416,0.835163,0.299075,NaN
4,0.755549,0.786832,0.790244,0.788534,0.834108,0.295652,NaN
mean,0.761332,0.776946,0.804902,0.790572,0.837381,0.295578,1.0
